## 问题一

### 数据分析处理

In [ ]:
import pandas as pd
import os

# 导入数据
data1=pd.read_excel('附件.xlsx',sheet_name='表单1')
data2=pd.read_excel('附件.xlsx',sheet_name='表单2')
print("描述性统计",data1.describe())
print(data1.isnull().sum())

# 定类定量化处理
temp=pd.DataFrame()
temp['纹饰']=data1['纹饰'].astype('category').cat.codes+1
temp['类型']=data1['类型'].astype('category').cat.codes+1
temp['颜色']=data1['颜色'].astype('category').cat.codes+1
temp['表面风化']=data1['表面风化'].astype('category').cat.codes+1

# spearman相关性分析
from spsspro.algorithm import descriptive_analysis
result = descriptive_analysis.correlation_analysis(temp[['纹饰', '类型', '颜色', '表面风化']],select_analysis= 'spearman')
print(result)

# 表2对文物重新编号，填充空值
data2['文物编号']=data2['文物采样点'].apply(lambda x:int(str(x)[:2]))
data2.fillna(0,inplace=True)

# 合并表1表2，存储数据
data_merge=pd.merge(data1,data2,on=['文物编号'])
data_merge.to_excel('表1表2合并.xlsx',index=None)

# 清洗数据，去除85-105之外的数据，并保存
data_merge['sum']=0
for i in ['二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']:
    data_merge['sum']+=data_merge[i]
data_merge=data_merge[(data_merge['sum']>85)&(data_merge['sum']<105)]
data_merge.reset_index(inplace=True,drop=True)
data_merge.to_excel('表1表2合并清洗.xlsx',index=None)

# 对定类数据变为虚拟变量
X=pd.get_dummies(data_merge[['纹饰', '类型', '颜色', '表面风化']])
X1=data_merge[data_merge['表面风化']=='风化'][['文物编号', '纹饰', '类型', '颜色', '表面风化']]
# 按照表面风化分类
X2=X.loc[data_merge[data_merge['表面风化']=='风化'].index][X.columns[:-1]]



### 风化预测

In [ ]:
# 岭回归
y=56.758+1.618 * X2['纹饰_A']+33.329 * X2['纹饰_B']-7.329 * X2['类型_铅钡']-12.447 * X2['纹饰_C']+0.9 * X2['颜色_浅蓝']-12.106 * X2['颜色_深蓝']-2.802 * X2['颜色_黑']+7.954 * X2['表面风化_无风化']-7.954 * 1 +3.602 * X2['颜色_绿']-8.901 * X2['颜色_紫']+0.466 * X2['颜色_蓝绿']+6.125 * X2['颜色_深绿']+10.405 * X2['颜色_浅绿']+7.329 * X2['类型_高钾']
X1['二氧化硅(SiO2)']=y.to_list()

# 逐步回归
y=0.428+0.999*X2['颜色_浅蓝']
X1['氧化钠(Na2O)']=y.to_list()

# 逐步回归
y=3.697-3.39*X2['类型_铅钡']-8.061*X2['纹饰_B']+7.087*X2['类型_高钾']-2.179*X2['颜色_蓝绿']
X1['氧化钾(K2O)']=y.to_list()
# 氧化钾(K2O)=3.483＋0.402 × 纹饰_A-5.002 × 纹饰_B-3.979 × 类型_铅钡＋1.24 × 纹饰_C＋0.595 × 颜色_浅蓝＋0.377 × 颜色_深蓝＋0.752 × 颜色_黑＋0.626 × 表面风化_无风化-0.626 × 表面风化_风化-1.211 × 颜色_绿-0.418 × 颜色_紫-0.685 × 颜色_蓝绿＋0.044 × 颜色_深绿-0.653 ×  X2['颜色_浅绿']＋3.979 × X2['类型_高钾']

# 逐步回归
y=2.05+3.283*X2['类型_高钾']-4.462*X2['纹饰_B']
X1['氧化钙(CaO)']=y.to_list()

# 逐步回归
y=0.43+0.605*X2['纹饰_A']
X1['氧化镁(MgO)']=y.to_list()

# 逐步回归
y=2.59+3.471*X2['纹饰_A']
X1['氧化铝(Al2O3)']=y.to_list()

# 逐步回归
y=0.642-0.128+0.77*X2['表面风化_无风化']
X1['氧化铁(Fe2O3)']=y.to_list()
# y=0.642-0.128*X2['表面风化_风化']+0.77*X2['表面风化_无风化']
# y=1.001+0.245 * X2['纹饰_A']-0.901 * X2['纹饰_B']+0.055 * X2['纹饰_C']-0.294 * X2['类型_铅钡']+0.294 * X2['类型_高钾']+0.127 * X2['颜色_浅绿']-0.085 * X2['颜色_浅蓝']-0.522 * X2['颜色_深绿'] * 0.199 * X2['颜色_深蓝']+0.076 * X2['颜色_紫']-0.892 * X2['颜色_绿']+0.227 * X2['颜色_蓝绿'] + 0.338 * X2['颜色_黑']

# 逐步回归
y=1.546+4.558*X2['颜色_紫']
X1['氧化铜(CuO)']=y.to_list()

# 逐步回归
y=5.127-9.688*X2['类型_高钾']+14.815*X2['类型_铅钡']+14.818*X2['纹饰_C']-3.378*X2['表面风化_无风化']+8.505+17.256*X2['颜色_深蓝']-11.592*X2['颜色_紫']
X1['氧化铅(PbO)']=y.to_list()
# y=5.127-9.688*X2['类型_高钾']+14.815*X2['类型_铅钡']+14.818*X2['纹饰_C']-3.378*X2['表面风化_无风化']+8.505*X2['表面风化_风化']+17.256*X2['颜色_深蓝']-11.592*X2['颜色_紫']
# 氧化铅(PbO)=16.843-4.128 × 纹饰_A-12.042 × 纹饰_B＋10.598 × 类型_铅钡＋7.946 × 纹饰_C-0.862 × 颜色_浅蓝＋14.233 × 颜色_深蓝-0.395 × 颜色_黑-6.635 × 表面风化_无风化＋6.635 × 表面风化_风化＋3.685 × 颜色_绿-8.819 × 颜色_紫＋0.62 × 颜色_蓝绿＋3.195 × 颜色_深绿-4.021 × 颜色_浅绿-10.598 × 类型_高钾

# 逐步回归
y=2.884+18.281*X2['颜色_紫']+5.368*X2['类型_铅钡']-2.485*X2['类型_高钾']
X1['氧化钡(BaO)']=y.to_list()
# 氧化钡(BaO)=5.823＋0.398 × 纹饰_A-2.406 × 纹饰_B＋4.181 × 类型_铅钡＋0.398 × 纹饰_C-2.896 × 颜色_浅蓝-0.106 × 颜色_深蓝-3.038 × 颜色_黑-0.615 × 表面风化_无风化＋0.615 × 表面风化_风化-1.625 × 颜色_绿＋14.206 × 颜色_紫＋0.441 × 颜色_蓝绿-1.977 × 颜色_深绿-3.038 × 颜色_浅绿-4.181 × 类型_高钾

#逐步回归
y=1.465+3.063-4.248*X2['纹饰_B']-2.683*X2['颜色_深绿']
X1['五氧化二磷(P2O5)']=y.to_list()
# y=1.465+3.063*X2['表面风化_风化']-4.248*X2['纹饰_B']-2.683*X2['颜色_深绿']

# 逐步回归
y=0.117-0.089*X2['类型_高钾']+0.206*X2['类型_铅钡']+0.207*X2['颜色_紫']
X1['氧化锶(SrO)']=y.to_list()

# 逐步回归
y=0.012+1.055*X2['颜色_深蓝']+0.25*X2['颜色_黑']
X1['氧化锡(SnO2)']=y.to_list()

# 逐步回归
y=0.08+5.84*X2['颜色_紫']
X1['二氧化硫(SO2)']=y.to_list()

# 数据正值处理
for i in ['二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']:
    X1[i]=X1[i].clip(lower=0)
    
# 验证是否在85-105之间
X1['sum']=0
for i in ['二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']:
    X1['sum']+=X1[i]
X1[(X1['sum']<85)|(X1['sum']>105)]

# 保存数据
X1.to_excel('风化物品未风化时化学成分含量.xlsx',index=None)

## 问题二

### K-means聚类分析

### 高钾分类

In [ ]:
K_data=data_merge[['类型','二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']]

# 按照类型分类
K_data[K_data['类型']=='高钾']

# K-means聚类肘部图
from sklearn.cluster import KMeans
from yellowbrick.cluster.elbow import kelbow_visualizer
jk=K_data[K_data['类型']=='高钾']
jk.reset_index(inplace=True,drop=True)
kelbow_visualizer(KMeans(random_state=5), jk[jk.columns[1:]], k=(2,12))

# 轮廓可视化
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
k_means = KMeans(n_clusters=4, random_state=5)
visualizer = SilhouetteVisualizer(k_means)
visualizer.fit(jk[jk.columns[1:]])
visualizer.poof()

# K-means聚类预测
k_means = KMeans(n_clusters=4, random_state=5)
k_means.fit(Xk[Xk.columns[1:]])
y_predict = k_means.predict(jk[jk.columns[1:]])
jk['分类']=y_predict
jk.to_excel('k-means高钾.xlsx',index=None)

#画图
import matplotlib as mpl
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
clf=KMeans(n_clusters=4, random_state=5)  
clf.fit(jk[jk.columns[1:]])
pre=clf.predict(jk[jk.columns[1:]])
pca=PCA(n_components=3)
newDate=pca.fit_transform(jk[jk.columns[1:]])
num, dim = Xk[Xk.columns[1:]].shape
x1=newDate[:,0]  
x2=newDate[:,1]
x3=newDate[:,2]
color = ['r','g','b','c'] # 颜色集
label = ('CaO≤5.64%, SiO2≤89.7%, Fe2O3≤2.38%', 'CaO>5.64%', 'CaO≤5.64%, SiO2≤89.7%, Fe2O3>2.38%', 'CaO≤5.64%, SiO2>89.7%')  # 图例说明集
fig = plt.figure(figsize=(8, 8),dpi=100)
plt.rcParams.update({'font.size':16})
ax=Axes3D(fig)
for p in range(0,num):
    y=pre[p]
    ax.scatter((newDate[p, 0]), (newDate[p, 1]), (newDate[p, 2]), c=color[(int(y))],s=100 )
y = np.arange(1, 5, 1)
y_unique = np.unique(y-1)
legend_lines = [mpl.lines.Line2D([0], [0], linestyle="none", marker='o', c=color[y]) for y in y_unique]
legend_labels = [label[y] for y in y_unique]
ax.legend(legend_lines, legend_labels, numpoints=1, title='高钾亚类化学成分',loc='upper left')
plt.show()

### 铅钡分类

In [ ]:
K_data=data_merge[['类型','二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']]

# 按照类型分类
K_data[K_data['类型']=='铅钡']

# K-means聚类肘部图
from sklearn.cluster import KMeans
from yellowbrick.cluster.elbow import kelbow_visualizer
jk=K_data[K_data['类型']=='铅钡']
jk.reset_index(inplace=True,drop=True)
kelbow_visualizer(KMeans(random_state=5), jk[jk.columns[1:]], k=(2,12))

# 轮廓可视化
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
k_means = KMeans(n_clusters=5, random_state=5)
visualizer = SilhouetteVisualizer(k_means)
visualizer.fit(jk[jk.columns[1:]])   
visualizer.poof()

# K-means聚类预测
k_means = KMeans(n_clusters=5, random_state=5)
k_means.fit(Xk[Xk.columns[1:]])
y_predict = k_means.predict(jk[jk.columns[1:]])
jk['分类']=y_predict
jk.to_excel('k-means铅钡.xlsx',index=None)

#画图
import matplotlib as mpl
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
clf=KMeans(n_clusters=4, random_state=5)  
clf.fit(jk[jk.columns[1:]])
pre=clf.predict(jk[jk.columns[1:]])
pca=PCA(n_components=3)
newDate=pca.fit_transform(jk[jk.columns[1:]])
num, dim = Xk[Xk.columns[1:]].shape
x1=newDate[:,0]  
x2=newDate[:,1]
x3=newDate[:,2]
color = ['r','g','b','c','y'] # 颜色集
label = ('PbO≤33.55%, SiO2≤32.765%', 'PbO≤33.55%, SiO2>57.665%', '33.55%<PbO≤50.33%', 'PbO>50.33%','PbO≤33.55%, SiO2>32.765%, CuO>0.22%')  # 图例说明集
fig = plt.figure(figsize=(8, 8),dpi=100)
plt.rcParams.update({'font.size':16})
ax=Axes3D(fig)
for p in range(0,num):
    y=pre[p]
    ax.scatter((newDate[p, 0]), (newDate[p, 1]), (newDate[p, 2]), c=color[(int(y))],s=100 )
y = np.arange(1, 6, 1)
y_unique = np.unique(y-1)
legend_lines = [mpl.lines.Line2D([0], [0], linestyle="none", marker='o', c=color[y]) for y in y_unique]
legend_labels = [label[y] for y in y_unique]
ax.legend(legend_lines, legend_labels, numpoints=1, title='铅钡亚类化学成分',loc='upper left')
plt.show()

## 问题三

### 整体模型预测

In [ ]:
# 读入并清洗填充数据
data_merge=data_merge[(data_merge['sum']>85)&(data_merge['sum']<105)]
data3=pd.read_excel('附件.xlsx',sheet_name='表单3')
data3.fillna(0,inplace=True)

# 对数据进行处理
data_new = pd.read_excel('表1表2去除未风化点.xlsx')
data_new=data_new[['类型','表面风化','二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']]
data_new=pd.concat([data_new,pd.get_dummies(data_new['表面风化'])],axis=1)
data_merge = data_new
X=data_merge[['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)', '无风化',
       '风化']]
y=data_merge['类型']

# 拆分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
    
#逻辑回归
model = LogisticRegression()
model.fit(x_train, y_train)
print('逻辑回归')
print(classification_report(model.predict(x_test),y_test))

# 随机森林分类
model = RandomForestClassifier()
model.fit(x_train, y_train)
print('随机森林分类')
print(classification_report(model.predict(x_test),y_test))

# lgbm分类
model = lgb.LGBMClassifier()
model.fit(x_train, y_train)
print('lgbm分类')
print(classification_report(model.predict(x_test),y_test))


# XGboost分类
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
model = xgb.XGBClassifier()
model.fit(x_train, y_train)
print('XGboost分类')
print(classification_report(model.predict(x_test),y_test))

### 使用XGBoost分类预测类型

In [ ]:
y = le.fit_transform(y)
model = xgb.XGBClassifier()
model.fit(X, y)
model.predict(data3[['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)', '无风化',
       '风化']])

### XGBoost灵敏性分析

In [ ]:
#取消科学计数法
import  pandas as pd
import numpy as np
pd.set_option("display.float_format", lambda x: "%.2f" % x) #为了直观的显示数字，不采用科学计数法
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# XGBoost n_estimators（基学习器数量）敏感度分析
y = le.fit_transform(y)
data_X=[]
data_y=[]
for i in range(1,1000,20):
    model = xgb.XGBClassifier(n_estimators=i,
    max_depth=6)
    model.fit(X, y)
    data_X.append(i)
    data_y.append(f1_score(y,model.predict(X), pos_label=1))
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('n_estimators')
plt.ylabel('f1')
plt.title('XGBoost n_estimators（基学习器数量）敏感度分析')
plt.legend()
plt.savefig('XGBoost分类 n_estimators（基学习器数量）敏感度分析.jpg')
plt.show()

# XGBoost max_depth（树最大深度）敏感度分析
data_X=[]
data_y=[]
y = le.fit_transform(y)
for i in range(1,50,2):
    model = xgb.XGBClassifier(n_estimators=50,
    max_depth=i)
    model.fit(X, y)
    data_X.append(i)
    data_y.append(f1_score(y,model.predict(X), pos_label=1))
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('max_depth')
plt.ylabel('f1')
plt.title('XGBoost max_depth（树最大深度）敏感度分析')
plt.legend()
plt.savefig('XGBoost分类 max_depth的敏感度分析.jpg')
plt.show()

# XGBoost learning_rate（学习率）的敏感度分析
data_X=[]
data_y=[]
for i in range(1,100,1):
    model = xgb.XGBClassifier(n_estimators=100,
    max_depth=6, learning_rate=i/100)
    model.fit(X, y)
    data_X.append(i/100)
    data_y.append(f1_score(y,model.predict(X), pos_label=1))
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('learning_rate')
plt.ylabel('f1')
plt.title('XGBoost learning_rate（学习率）的敏感度分析')
plt.legend()
plt.savefig('XGBoost分类 learning_rate（学习率）的敏感度分析.jpg')
plt.show()

# XGBoost reg_lambda（L2正则化项）的敏感度分析
data_X=[]
data_y=[]
for i in range(1,1000,2):
    model = xgb.XGBClassifier(n_estimators=100,
    max_depth=6, learning_rate=0.1,reg_lambda=i/1000)
    model.fit(X, y)
    data_X.append(i/1000)
    data_y.append(f1_score(y,model.predict(X), pos_label=1))
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('reg_lambda')
plt.ylabel('f1')
plt.title('XGBoost reg_lambda（L2正则化项）的敏感度分析')
plt.legend()
plt.savefig('XGBoost分类 reg_lambda（L2正则化项）的敏感度分析.jpg')
plt.show()




### LightGBM灵敏性分析

In [ ]:
X=data_merge[['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)', '无风化',
       '风化']]
y=data_merge['类型']

# LightGBM n_estimators（基学习器数量）敏感度分析
data_X=[]
data_y=[]
for i in range(1,20,1):
    model = lgb.LGBMClassifier(n_estimators=i,
    max_depth=None)
    model.fit(X, y)
    data_X.append(i)
    data_y.append(f1_score(y,model.predict(X), pos_label='高钾'))
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('n_estimators')
plt.ylabel('f1')
plt.title('LightGBM n_estimators（基学习器数量）敏感度分析')
plt.legend()
plt.savefig('LightGBM分类 n_estimators（基学习器数量）敏感度分析.jpg')
plt.show()

# LightGBM max_depth（树最大深度）敏感度分析
data_X=[]
data_y=[]
for i in range(1,50,2):
    model = lgb.LGBMClassifier(n_estimators=50,
    max_depth=i)
    model.fit(X, y)
    data_X.append(i)
    data_y.append(f1_score(y,model.predict(X), pos_label='高钾'))
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('max_depth')
plt.ylabel('f1')
plt.title('LightGBM max_depth（树最大深度）敏感度分析')
plt.legend()
plt.savefig('LightGBM分类 max_depth 的敏感度分析.jpg')
plt.show()

# LightGBM learning_rate（学习率）的敏感度分析
data_new = pd.read_excel('表1表2合并清洗.xlsx')
data_new=pd.concat([data_new,pd.get_dummies(data_new['表面风化'])],axis=1)
data_new=data_new[['类型','表面风化','二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']]
data_new=pd.concat([data_new,pd.get_dummies(data_new['表面风化'])],axis=1)
data_merge = data_new
X=data_merge[['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)', '无风化',
       '风化']]
y=data_merge['类型']
data_X=[]
data_y=[]
for i in range(1,10,1):
    model = lgb.LGBMClassifier(n_estimators=100,
    max_depth=6, learning_rate=i/100)
    model.fit(X, y)
    data_X.append(i/100)
    data_y.append(f1_score(y,model.predict(X), pos_label='高钾'))
#     print(i)
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('learning_rate')
plt.ylabel('f1')
plt.title('LightGBM learning_rate（学习率）的敏感度分析')
plt.legend()
plt.savefig('LightGBM分类 learning_rate（学习率）的敏感度分析.jpg')
plt.show()

# LightGBM reg_lambda（L2正则化项）的敏感度分析
data_X=[]
data_y=[]
y=data_merge['类型']
for i in range(1,1000,2):
    model = lgb.LGBMClassifier(n_estimators=100,
    max_depth=6, learning_rate=0.1,reg_lambda=i/1000)
    model.fit(X, y)
    data_X.append(i/1000)
    data_y.append(f1_score(y,model.predict(X), pos_label='高钾'))
#     print(i)
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(data_X,data_y)
plt.xlabel('reg_lambda')
plt.ylabel('f1')
plt.title('LightGBM reg_lambda（L2正则化项）的敏感度分析')
plt.legend()
plt.savefig('LightGBM分类 reg_lambda（L2正则化项）的敏感度分析.jpg')
plt.show()

### 交叉验证算法模型稳定性

In [ ]:
from sklearn.model_selection import cross_val_score
data_new = pd.read_excel('表1表2去除未风化点.xlsx')
data_new=pd.concat([data_new,pd.get_dummies(data_new['表面风化'])],axis=1)
data_new=data_new[['类型','表面风化','二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']]、
data_new=pd.concat([data_new,pd.get_dummies(data_new['表面风化'])],axis=1)
data_merge = data_new
X=data_merge[['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)', '无风化',
       '风化']]
y=data_merge['类型']
# 拆分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
    
yx=e.fit_transform(y)
xg_l = []
lg_l = []
for i in range(10):
    xg = xgb.XGBClassifier(n_estimators=25)
    lg = lgb.LGBMClassifier(n_estimators=25)
    xg_s = cross_val_score(xg,X,yx,cv=10).mean()
    xg_l.append(xg_s)
    lg_s = cross_val_score(lg,X,y,cv=10).mean()
    lg_l.append(lg_s)
plt.figure(figsize=(8, 4),dpi=100)
plt.plot(range(1,11),xg_l,'r-',label = "XGBoost",markersize=10)
plt.plot(range(1,11),lg_l,'b-.',label = "LightGBM",markersize=10)
plt.ylabel('准确度')
plt.ylim([0.9,1.03])
plt.title('XGBoost分类 & LightGBM分类交叉验证对比图')
plt.legend()
plt.savefig('XGBoost分类 & LightGBM分类交叉验证对比图.jpg')
plt.show()

## 问题四

### 数据处理

In [ ]:
data_merge[['类型',  '二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)']].to_excel('4.xlsx',index=None)
data_merge[data_merge['类型']=='高钾'][['类型',  '二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)']].to_excel('4高钾.xlsx',index=None)
data_merge[data_merge['类型']=='铅钡'][['类型',  '二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)',
       '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)',
       '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)']].to_excel('4铅钡.xlsx',index=None)

### K-means++聚类分析并画图

In [ ]:
data_merge = data_merge[['类型','表面风化','二氧化硅(SiO2)', '氧化钠(Na2O)',
       '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)',
       '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)',
       '氧化锡(SnO2)', '二氧化硫(SO2)']]
gj = data_merge[data_merge['类型']=='高钾']
qb = data_merge[data_merge['类型']=='铅钡']
gj_wf = gj[gj['表面风化']=='无风化']
gj_f = gj[gj['表面风化']=='风化']
qb_wf = qb[qb['表面风化']=='无风化']
qb_f = qb[qb['表面风化']=='风化']

from sklearn.cluster import KMeans
from yellowbrick.cluster.elbow import kelbow_visualizer

# 高钾无风化分类
gj_wf.reset_index(inplace=True,drop=True)
kelbow_visualizer(KMeans(random_state=5), gj_wf[gj_wf.columns[2:]], k=(2,12))
k_means = KMeans(n_clusters=5, random_state=5)
k_means.fit(gj_wf[gj_wf.columns[2:]])
y_predict = k_means.predict(gj_wf[gj_wf.columns[2:]])
gj_wf['分类'] = y_predict
gj_wf.to_excel('gj_wf.xlsx')
fig = plt.figure(figsize=(8, 8),dpi=100)
colors = cm.GnBu(np.arange(5)/5) 
plt.pie([3,2,1,1,5],
        labels=['聚类1','聚类2','聚类3','聚类4','聚类5'], # 设置饼图标签
        colors=colors, # 设置饼图颜色
        shadow=True,
        autopct="%1.2f%%"
       )
plt.title("高钾未风化聚类饼图") # 设置标题
plt.show()

# 高钾风化分类
gj_f.reset_index(inplace=True,drop=True)
# Use the quick method and immediately show the figure
kelbow_visualizer(KMeans(random_state=5), gj_f[gj_f.columns[2:]], k=(2, 7))
k_means = KMeans(n_clusters=4, random_state=5)
k_means.fit(gj_wf[gj_f.columns[2:]])
y_predict = k_means.predict(gj_f[gj_f.columns[2:]])
gj_f['分类'] = y_predict
gj_f.to_excel('gj_f.xlsx')
fig = plt.figure(figsize=(8, 8),dpi=100)
colors = cm.GnBu(np.arange(1)/1) 
plt.pie([1],
        labels=['聚类1'], # 设置饼图标签
        colors=colors, # 设置饼图颜色
        shadow=True,
        autopct="%1.2f%%"
       )
plt.title("高钾风化聚类饼图") # 设置标题
plt.show()

# 铅钡未风化分类
qb_wf.reset_index(inplace=True,drop=True)
# Use the quick method and immediately show the figure
kelbow_visualizer(KMeans(random_state=5), qb_wf[qb_wf.columns[2:]], k=(2, 12))
k_means = KMeans(n_clusters=5, random_state=5)
k_means.fit(qb_wf[qb_wf.columns[2:]])
y_predict = k_means.predict(qb_wf[qb_wf.columns[2:]])
qb_wf['分类'] = y_predict
qb_wf.to_excel('qb_wf.xlsx')
fig = plt.figure(figsize=(8, 8),dpi=100)
colors = cm.GnBu(np.arange(5)/5) 
plt.pie([6,3,1,2,1],
        labels=['聚类1','聚类2','聚类3','聚类4','聚类5'], # 设置饼图标签
        colors=colors, # 设置饼图颜色
        shadow=True,
        autopct="%1.2f%%"
       )
plt.title("铅钡未风化聚类饼图") # 设置标题
plt.show()

# 铅钡风化分类
qb_f.reset_index(inplace=True,drop=True)
# Use the quick method and immediately show the figure
kelbow_visualizer(KMeans(random_state=5), qb_f[qb_f.columns[2:]], k=(2, 12))
k_means = KMeans(n_clusters=4, random_state=5)
k_means.fit(qb_f[qb_f.columns[2:]])
y_predict = k_means.predict(qb_f[qb_f.columns[2:]])
qb_f['分类'] = y_predict
qb_f.to_excel('qb_f.xlsx')
fig = plt.figure(figsize=(8, 8),dpi=100)
colors = cm.GnBu(np.arange(4)/4) 
plt.pie([15,11,4,6],
        labels=['聚类1','聚类2','聚类3','聚类4'], # 设置饼图标签
        colors=colors, # 设置饼图颜色
        shadow=True,
        autopct="%1.2f%%"
       )
plt.title("铅钡风化聚类饼图") # 设置标题
plt.show()